In [71]:
import sys
tools_path = os.path.join('..', 'tools')
sys.path.append(tools_path)
import file_tools

import os
_DF_DIR = os.path.join('.', 'df_files')
_FINAL_DFS_DIR = os.path.join('.', 'final_dfs')
file_tools.ensure_dir(_FINAL_DFS_DIR)

In [72]:
import numpy as np
import pandas as pd

In [73]:
dfs = {}
for f in file_tools.list_files(_DF_DIR, '*.parquet'):
    dbf = os.path.join(_DF_DIR, f)
    df = pd.read_parquet(dbf, engine='pyarrow')
    basename = file_tools.get_filebasename(f)
    dfs[basename] = df
    print(basename, len(df), df['method_name'].unique())

results_cluster 23836 ['LinearRegression' 'SVR' 'MLP']
results_latest_moresvr 23836 ['LinearRegression' 'SVR' 'MLP']
optim_ablation 1352500 ['LinearRegression']
results_clustersvr 340 ['BestSVR']
results_dimensionality_eeg 10346 ['LinearRegression' 'Shuffle_LinearRegression' 'DummyScoring_Mean']
results_dimensionality_face 10200 ['LinearRegression' 'Shuffle_LinearRegression' 'DummyScoring_Mean']
results 24592 ['LinearRegression' 'Shuffle_LinearRegression' 'DummyScoring_Mean' 'SVR'
 'RandomForest' 'MLP']
results_ablation 36720 ['LinearRegression' 'Shuffle_LinearRegression' 'DummyScoring_Mean']
optim 800000 ['LinearRegression' 'Shuffle_LinearRegression' 'DummyScoring_Mean']
results_best_again 371 ['BestSVR' 'BestMLP']
results_best_cluster 371 ['BestSVR' 'BestMLP']


In [81]:
# some local regressors runs were test run incomplete to prepare for cluster runs
mask_results = ((dfs['results']['method_name'] != "SVR") &
        (dfs['results']['method_name'] != "MLP") &
        (dfs['results']['method_name'] != "RandomForest"))

result_df = dfs['results'][mask_results]
print(result_df['method_name'].unique())

# LinReg on cluster is a duplicate, we could keep cluster one or local run one
# we remove cluste one
mask_results_cluster = ((dfs['results_cluster']['method_name'] != "LinearRegression"))

results_cluster_df = dfs['results_cluster'][mask_results_cluster]
print(results_cluster_df['method_name'].unique())

results_best_cluster_df = dfs['results_best_cluster']

final_result_df = pd.concat([result_df, results_cluster_df, results_best_cluster_df], axis=0)
print(final_result_df['method_name'].unique())

['LinearRegression' 'Shuffle_LinearRegression' 'DummyScoring_Mean']
['SVR' 'MLP']
['LinearRegression' 'Shuffle_LinearRegression' 'DummyScoring_Mean' 'SVR'
 'MLP' 'BestSVR' 'BestMLP']


In [82]:
results_filename = os.path.join(_FINAL_DFS_DIR, 'results.parquet')
final_result_df.to_parquet(results_filename, engine='pyarrow')

results_ablation_filename = os.path.join(_FINAL_DFS_DIR, 'results_ablation.parquet')
dfs['results_ablation'].to_parquet(results_ablation_filename, engine='pyarrow')

In [83]:
results_dimensionality_eeg_filename = os.path.join(_FINAL_DFS_DIR, 'results_dimensionality_eeg.parquet')
dfs['results_dimensionality_face'].to_parquet(results_dimensionality_eeg_filename, engine='pyarrow')

results_dimensionality_face_filename = os.path.join(_FINAL_DFS_DIR, 'results_dimensionality_face.parquet')
dfs['results_dimensionality_face'].to_parquet(results_dimensionality_face_filename, engine='pyarrow')

In [84]:
optim_filename = os.path.join(_FINAL_DFS_DIR, 'optim.parquet')
dfs['optim'].to_parquet(optim_filename, engine='pyarrow')

optim_ablation_filename = os.path.join(_FINAL_DFS_DIR, 'optim_ablation.parquet')
dfs['optim_ablation'].to_parquet(optim_ablation_filename, engine='pyarrow')